In [3]:
import numpy as np
import cv2
from imutils import paths
import cv2 as cv

In [19]:
cap = cv2.VideoCapture(0)
cap2 = cv2.VideoCapture(2)


if not cap.isOpened() or not cap2.isOpened():
    print("Cannot open camera")
    exit()
count = 0
while True:
    # Capture frame-by-frame
    ret, frameL = cap.read()
    ret2, frameR = cap2.read()
    
    
    sift = cv.SIFT_create()
    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(frameL, None)
    kp2, des2 = sift.detectAndCompute(frameR, None)
    
    imgSift = cv.drawKeypoints(frameL, kp1, None, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    cv.imshow("SIFT Keypoints", imgSift)
    
    # Match keypoints in both images
    # Based on: https://docs.opencv.org/master/dc/dc3/tutorial_py_matcher.html
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)   # or pass empty dictionary
    flann = cv.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    # Keep good matches: calculate distinctive image features
    # Lowe, D.G. Distinctive Image Features from Scale-Invariant Keypoints. International Journal of Computer Vision 60, 91–110 (2004). https://doi.org/10.1023/B:VISI.0000029664.99615.94
    # https://www.cs.ubc.ca/~lowe/papers/ijcv04.pdf
    matchesMask = [[0, 0] for i in range(len(matches))]
    good = []
    pts1 = []
    pts2 = []

    for i, (m, n) in enumerate(matches):
        if m.distance < 0.7*n.distance:
            # Keep this keypoint pair
            matchesMask[i] = [1, 0]
            good.append(m)
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)
    
    
    # Draw the keypoint matches between both pictures
    # Still based on: https://docs.opencv.org/master/dc/dc3/tutorial_py_matcher.html
    draw_params = dict(matchColor=(0, 255, 0),
                       singlePointColor=(255, 0, 0),
                       matchesMask=matchesMask[300:500],
                       flags=cv.DrawMatchesFlags_DEFAULT)

    keypoint_matches = cv.drawMatchesKnn(
        frameL, kp1, frameR, kp2, matches[300:500], None, **draw_params)
    cv.imshow("Keypoint matches", keypoint_matches)
    
    pts1 = np.int32(pts1)
    pts2 = np.int32(pts2)
    img1 = frameL
    img2 = frameR
    print(pts1, pts2)
    if len(pts1) and len(pts2) > 0:
        fundamental_matrix, inliers = cv.findFundamentalMat(pts1, pts2, cv.FM_RANSAC)
        
        if fundamental_matrix is not None and fundamental_matrix.shape == (3,3):
    
            # We select only inlier points
            pts1 = pts1[inliers.ravel() == 1]
            pts2 = pts2[inliers.ravel() == 1]


            def drawlines(img1src, img2src, lines, pts1src, pts2src):
                ''' img1 - image on which we draw the epilines for the points in img2
                    lines - corresponding epilines '''
                r, c = img1src.shape
                img1color = cv.cvtColor(img1src, cv.COLOR_GRAY2BGR)
                img2color = cv.cvtColor(img2src, cv.COLOR_GRAY2BGR)
                # Edit: use the same random seed so that two images are comparable!
                np.random.seed(0)
                for r, pt1, pt2 in zip(lines, pts1src, pts2src):
                    color = tuple(np.random.randint(0, 255, 3).tolist())
                    x0, y0 = map(int, [0, -r[2]/r[1]])
                    x1, y1 = map(int, [c, -(r[2]+r[0]*c)/r[1]])
                    img1color = cv.line(img1color, (x0, y0), (x1, y1), color, 1)
                    img1color = cv.circle(img1color, tuple(pt1), 5, color, -1)
                    img2color = cv.circle(img2color, tuple(pt2), 5, color, -1)
                return img1color, img2color



            # Find epilines corresponding to points in right image (second image) and
            # drawing its lines on left image
            lines1 = cv.computeCorrespondEpilines(
                pts2.reshape(-1, 1, 2), 2, fundamental_matrix)
            lines1 = lines1.reshape(-1, 3)
            img5, img6 = drawlines(img1, img2, lines1, pts1, pts2)

            # Find epilines corresponding to points in left image (first image) and
            # drawing its lines on right image
            lines2 = cv.computeCorrespondEpilines(
                pts1.reshape(-1, 1, 2), 1, fundamental_matrix)
            lines2 = lines2.reshape(-1, 3)
            img3, img4 = drawlines(img2, img1, lines2, pts2, pts1)

            plt.subplot(121), plt.imshow(img5)
            plt.subplot(122), plt.imshow(img3)
            plt.suptitle("Epilines in both images")
            plt.show()
    
    
    
    
    
    
    
    
    # Display the resulting frame
    cv2.imshow('Stereo 1', frameL)
    cv2.imshow('stereo 2',frameR)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break
    if key == ord('s'):
        print("Saving image")
        cv2.imwrite("./testData/"+str(count)+"L.jpg", frameL)
        cv2.imwrite("./testData/"+str(count)+"R.jpg", frameR)
        count = count+1
# When everything done, release the capture
cap.release()
cap2.release()
cv2.destroyAllWindows()

[] []
[[ 22 418]
 [ 30 432]
 [ 30 432]
 [ 31 436]
 [ 31 436]
 [ 36 439]
 [ 37 433]
 [ 37 433]
 [ 41 444]
 [ 65 353]
 [ 72 354]
 [ 95 408]
 [101 402]
 [101 335]
 [102 339]
 [103 352]
 [103 370]
 [103 323]
 [104 390]
 [104 302]
 [104 337]
 [113 306]
 [123 355]
 [123 423]
 [127 274]
 [127 363]
 [127 363]
 [135 356]
 [136 374]
 [142 371]
 [144 327]
 [145 358]
 [148 394]
 [152 422]
 [154 410]
 [156 329]
 [157 352]
 [158 357]
 [160 363]
 [163 383]
 [170 474]
 [171 336]
 [171 377]
 [172 257]
 [175 452]
 [175 347]
 [179 422]
 [179 353]
 [182 355]
 [184 344]
 [187 332]
 [188 340]
 [188 340]
 [194 252]
 [201 379]
 [203 451]
 [215 277]
 [216 314]
 [216 258]
 [217 377]
 [222 439]
 [222 296]
 [227 255]
 [227 455]
 [228 391]
 [229 416]
 [230 457]
 [234 314]
 [234 322]
 [234 417]
 [234 407]
 [237 378]
 [238 442]
 [240 430]
 [241 409]
 [241 275]
 [242 402]
 [244 422]
 [244 438]
 [244 438]
 [245 442]
 [246 332]
 [247 321]
 [249 404]
 [250 331]
 [251 375]
 [254 390]
 [255 443]
 [258 386]
 [260 412]
 [26

ValueError: too many values to unpack (expected 2)

In [20]:
# When everything done, release the capture
cap.release()
cap2.release()
cv2.destroyAllWindows()